# **Task 1.1 - Download Files Scipt**
Recieve 10-K Files from Adobe, Intel Corporation and Microsoft Corp

In [79]:
# **Task 1.1 - Download Files Scipt**
Recieve 10-K Files from Adobe, Intel Corporation and Microsoft Corp

In [7]:
from sec_edgar_downloader import Downloader
import os
from bs4 import BeautifulSoup

dl = Downloader("Company Name", "my.email@domain.com")
companies = ["ADBE", "INTC", "MSFT"]
subdirectories = []

FileNotFoundError: [WinError 3] The system cannot find the path specified: '10-K Files/ADBE'

In [80]:
for company in companies:
    dl.get("10-K", company, after = "1995-01-01", before = "2024-01-01")
   
    org_path = "sec-edgar-filings/"+company+"/10-K/"

    for fileName in os.listdir(org_path):
        if fileName.startswith("000"):
            newName = fileName[11:]
            if (newName[0] == '9'):
                newName = "19" + newName 
            else:
                newName = "20" + newName
            path = org_path + fileName
            new_path= org_path + newName
            if (os.path.exists(new_path)):
                os.remove(path+"/full-submission.txt")
                os.rmdir(path)
            else:
                os.rename(path, new_path)

    temp = [d for d in os.listdir(org_path) if os.path.isdir(os.path.join(org_path, d))]
    for i in range(len(temp)):
        subdirectories.append(temp[i])
    os.mkdir("10-K Files/"+company)

# make all files readable for interpretation
companyNum = 0
year = 1995
fileName = "/full-submission.txt"
for index, subdirectory in enumerate(subdirectories):
    if (index % 29 == 0 and index != 0):
        companyNum = companyNum + 1
    path = "sec-edgar-filings/"+companies[companyNum]+"/10-K/"+subdirectory+fileName
    # Open and read the content of file
    with open(path, 'r', encoding='utf-8') as file:
        filing_content = file.read()

    soup = BeautifulSoup(filing_content, 'html.parser')

    # Extract the entire text content
    entire_content = soup.get_text()

    #Save the content to a text file with a unique name
    output_file_name = f"/10k_{year}.txt"  # Unique name based on index
    if (year + 1 < 2024):
        year += 1
    else:
        year = 1995
    output_path = "10-K Files/"+companies[companyNum]
    output_file_path = output_path + output_file_name
    
    with open(output_file_path, 'w', encoding='utf-8') as out_file:
        out_file.write(entire_content)

#     print(f"Extracted and saved content from 10-K file {index + 1} to: {output_file_path}")

Error occurred while downloading filing for accession number {}: {} 0001032210-00-001961 404 Client Error: Not Found for url: https://www.sec.gov/Archives/edgar/data/789019/000103221000001961/0001.txt
